In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader
from data import data_load_and_process
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchsummary import summary

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self, input_channels=1):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)  # Output size (batch, 32, 1)
        self.fc = nn.Linear(32, 1)

    def forward(self, x):
        # x shape: (batch, channels, length)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x


In [3]:

model = SimpleCNN(input_channels=1)  # or 3
summary(model, (1,10))  # or (3, 4, 4)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 10]              64
            Conv1d-2               [-1, 32, 10]           1,568
 AdaptiveAvgPool1d-3                [-1, 32, 1]               0
            Linear-4                    [-1, 1]              33
Total params: 1,665
Trainable params: 1,665
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.01
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [16]:


# Get data from data.py (choose your generator and reduction method)
# Example: 4x4 pseudo-image data, no reduction, 3 channels (change as needed)
q_num = 8 # Number of features/channels for reduction (set as needed)
X_train, Y_train, X_test, Y_test, DataID = data_load_and_process(
    q_num=q_num,
    data_gen='4x4_img_data',   # or 'sklearn_make_class', 'capital1_synthetic_data', '4x4_img_data'
    data_redu='tsne'        # or 'pca', 'autoencode', 'svd', 'tsne', 'no_redu'
)

print(len(X_train))
# Convert to torch tensors and reshape for 1D CNN
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)

if X_train.ndim == 2:
    # (N, L) -> (N, 1, L)
    X_train = X_train.unsqueeze(1)
    input_channels = 1
elif X_train.ndim == 3:
    # (N, C, L)
    input_channels = X_train.shape[1]
elif X_train.ndim == 4:
    # (N, C, H, W) -> (N, C, H*W)
    N, C, H, W = X_train.shape
    X_train = X_train.view(N, C, H * W)
    input_channels = C

train_dataset = TensorDataset(X_train, Y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Now you can use train_loader in your training loop


4000


In [17]:
# Hyperparameters
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN(input_channels=input_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.float().to(device).view(-1, 1)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")


Epoch 1/10, Loss: 0.6621
Epoch 2/10, Loss: 0.6246
Epoch 3/10, Loss: 0.5339
Epoch 4/10, Loss: 0.5169
Epoch 5/10, Loss: 0.4397
Epoch 6/10, Loss: 0.3665
Epoch 7/10, Loss: 0.3003
Epoch 8/10, Loss: 0.1569
Epoch 9/10, Loss: 0.2753
Epoch 10/10, Loss: 0.2215


In [18]:
# Prepare test DataLoader
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_test = torch.tensor(Y_test, dtype=torch.float32)
if X_test.ndim == 2:
    input_channels = 1
    length = X_test.shape[1]
    X_test = X_test.view(-1, input_channels, length)
test_dataset = TensorDataset(X_test, Y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).int().cpu().numpy().flatten()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy().flatten())

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")
print(f"Test F1 Score: {f1:.4f}")

Test Accuracy: 0.5330
Test Precision: 0.5263
Test Recall: 0.5274
Test F1 Score: 0.5268
